import pyarrow
pyarrow.__version__# Text pipeline
Text Pipeline: 

Current sentence + Emotion + context + NER + keyword  --[T5-fintuned-CommonGen]--> next sentence

In [68]:
import pyarrow
pyarrow.__version__

'8.0.0'

## Current sentence

In [22]:
import pandas as pd
storyData = pd.read_csv('../../data/csv_version/dev/emotion/allcharlinepairs-clean-v1.csv')
sentenceIndex = storyData[storyData['context'].isna()].sample()['sentence'].index
demoStory = storyData.iloc[sentenceIndex[0]:sentenceIndex[0]+5]
demoSentence = demoStory.iloc[0]['sentence']
print(demoSentence)
demoStory.style.set_properties(subset=['plutchik'], **{'width-min': '300px'})


Billy was feeling tired and had to get up to go to school.


,Unnamed: 0,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
5318,5318,455714e1-9419-4d9b-91e8-8ffa0005dae0,1,Billy,ann0,nan,Billy was feeling tired and had to get up to go to school.,yes,"['sleepy', 'tired', 'sad', 'sick']","{'joy': 0, 'trust': 0, 'fear': 1, 'surprise': 0, 'sadness': 2, 'disgust': 0, 'anger': 0, 'anticipation': 0}"
5319,5319,455714e1-9419-4d9b-91e8-8ffa0005dae0,2,Billy,ann0,Billy was feeling tired and had to get up to go to school.,On the way to school Billy stopped by the local convenience store.,yes,['tired'],"{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': 0, 'sadness': 1, 'disgust': 0, 'anger': 0, 'anticipation': 0}"
5320,5320,455714e1-9419-4d9b-91e8-8ffa0005dae0,3,Billy,ann0,Billy was feeling tired and had to get up to go to school.|On the way to school Billy stopped by the local convenience store.,As Billy entered he noticed an advertisement for an energy drink.,yes,"['curious', 'intrigued', 'eager']","{'joy': 1, 'trust': 0, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 1}"
5321,5321,455714e1-9419-4d9b-91e8-8ffa0005dae0,4,Billy,ann0,Billy was feeling tired and had to get up to go to school.|On the way to school Billy stopped by the local convenience store.|As Billy entered he noticed an advertisement for an energy drink.,Billy decided to buy the energy drink for himself.,yes,"['excited', 'determined']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 0, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}"
5322,5322,455714e1-9419-4d9b-91e8-8ffa0005dae0,5,Billy,ann0,Billy was feeling tired and had to get up to go to school.|On the way to school Billy stopped by the local convenience store.|As Billy entered he noticed an advertisement for an energy drink.|Billy decided to buy the energy drink for himself.,Billy swallowed the energy drink contents and felt awake for school.,yes,['alert'],"{'joy': 1, 'trust': 0, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 3}"


## Emotion

In [71]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np

# joy, trust, fear, surprise, sadness, disgust, anger, anticipation
emotionActivate =  {'joy' : True, 
                    'trust' : True, 
                    'fear' : True, 
                    'surprise' : True, 
                    'sadness' : True, 
                    'disgust' : True, 
                    'anger' : True, 
                    'anticipation' : True}

import torch
def genEmo(demoSentenceInput):
    model = AutoModelForSequenceClassification.from_pretrained("./firstSentenceDetection/")
    # assert model.config.output_attentions == True
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    
    encoding = tokenizer(demoSentenceInput, return_tensors="pt")
    # tokenizer.decode(encoding['input_ids'])
    encoding = {k: v.to('cpu') for k,v in encoding.items()}

    outputs = model(**encoding)
    
    # apply sigmoid + threshold
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= 0.5)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [list(emotionActivate.keys())[idx] for idx, label in enumerate(predictions) if label == 1.0 and list(emotionActivate.values())[idx] == True]
    return ", ".join( key for key in predicted_labels)

In [72]:
genEmo("I am happy")

'joy, trust, anticipation'

## NER

In [73]:
!pip install SceneGraphParser
!python -m spacy download en  
!pip install spacy

     ---------------------------------------- 12.8/12.8 MB 5.6 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [74]:
import sng_parser
def genKeyword(demoSentenceInput):
    graph = sng_parser.parse(demoSentenceInput)
    majorKeyword = [x['span'] for x in graph['entities']]
    return ", ".join( key for key in majorKeyword)

In [75]:
genKeyword("I and Ann were having a night walk yesterday")

'I, Ann, a night walk'

## Image Generation

In [76]:
demoStory.iloc[4]['sentence']

'May went home satisfied that she had spotted some wildlife.'

# T5 Sentence Generation

In [77]:
# Input Construction
def tempGetInput(currentSentenceindex):
    
    return f"{genKeyword(demoStory.iloc[currentSentenceindex]['sentence'])} | {demoStory.iloc[currentSentenceindex]['context'].replace('|', ' ')} | {genEmo(demoStory.iloc[currentSentenceindex]['sentence'])}" 

In [78]:
import transformers
import evaluate
from transformers import T5Tokenizer, T5ForConditionalGeneration
# ("mrm8488/t5-base-finetuned-common_gen")

model = T5ForConditionalGeneration.from_pretrained("./storyGen")
tokenizer = T5Tokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")


def genSentence(words, max_length=32):
    input_text = words
    features = tokenizer([input_text], return_tensors='pt')
    output = model.generate(input_ids=features['input_ids'], 
                            attention_mask=features['attention_mask'],
                            max_length=max_length)

    return tokenizer.decode(output[0], skip_special_tokens=True)

def genStory(storyLength = 5):
    story = []
    story.append(demoSentence);
    for i in range(1, storyLength):
        story.append(genSentence(tempGetInput(i)))
    return story

def BLEU_metric(generatedText, actualText):
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions= generatedText, references= actualText)
    return results


In [79]:
def getBLEUEachSentence(actual, expected):
    return [BLEU_metric([x], [y])['bleu'] for x, y in zip(list(actual), list(expected))]

def getMeteorEachSentence(actual, expected):
    return  [meteor([word_tokenize(x)], word_tokenize(y)) for x, y in zip(list(actual), list(expected))]

In [80]:
from pandas import DataFrame
from nltk.translate import meteor
from nltk import word_tokenize

testDatasetProcessed = DataFrame({"generated" : genStory(), "Ground truth": demoStory['sentence'].tolist(), 
                                  "BLEU" : getBLEUEachSentence(testDatasetProcessed["generated"], testDatasetProcessed["Ground truth"]), 
                                 "METEOR": getMeteorEachSentence(testDatasetProcessed["generated"], testDatasetProcessed["Ground truth"])})
# TO-DO: add score to dataframe
testDatasetProcessed.style.set_properties(subset=['generated'], **{'width-min': '300px'})


,generated,Ground truth,BLEU,METEOR
0,Someone had left a pair of binoculars on the park's bench.,Someone had left a pair of binoculars on the park's bench.,1.000000,0.999314
1,the binoculars were placed on the bench.,May happened upon the binoculars and sat on the bench to investigate.,0.390113,0.920940
2,"A few minutes later, the binoculars were full of birds.",She looked through the binoculars and searched around for birds.,0.483545,0.817901
3,"A few minutes later, she found that she had found the binoculars on the bench.","When she was finished, she returned the binoculars to the bench.",0.000000,0.312500
4,"A few minutes later, May saw that she had seen some wildlife.",May went home satisfied that she had spotted some wildlife.,0.000000,0.138889


[1.0, 0.0, 0.0, 0.4111336169005197, 0.5578002860768766]

[0.9985422740524781,
 0.49811613475177297,
 0.42592592592592593,
 0.5948372615039282,
 0.8294209702660407]

In [54]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\JAM_0\AppData\Roaming\nltk_data...


True